In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
df0 = pd.read_csv("Insurance_details_US_Central_day0.csv")
df1 = pd.read_csv("Insurance_details_US_Central_day1.csv")
df2 = pd.read_csv("Insurance_details_US_Central_day2.csv")

In [10]:
df0.head()

,Customer ID,Customer Title,Customer First Name,Customer Middle Name,Customer Last Name,Customer_Segment,Maritial_Status,Gender,DOB,Effective_Start_Dt,...,Policy_End_Dt,Next_Premium_Dt,Actual_Premium_Paid_Dt,Country,Region,State or Province,City,Postal Code,Total_Policy_Amt,Premium_Amt_Paid_TillDate
0,1354,Mr.,AARON,NaN,Dillon,Consumer,Single,Male,2/5/1997,12/31/2007,...,12/27/2036,3/31/2011,9/25/2011,U.S.A,Central,Texas,Weatherford,76086,500000,60000
1,697,Mister.,Adam,G,Sawyer,Corporate,Married,Male,2/5/1997,6/30/2011,...,1/1/2112,6/30/2012,12/31/2012,USA,Central,Indiana,East Chicago,46312,400000,120000
2,674,Mr,AlBERT,NaN,FROST,Small Business,Divorced,Male,2/6/1993,12/31/2009,...,1/1/2112,3/31/2013,9/25/2013,USA,Central,Missouri,Raytown,64133,500000,60000
3,188,Mrs.,ALEX,NaN,harrell,Corporate,Single,Male,2/5/1997,12/31/2010,...,12/27/2036,3/31/2014,9/25/2014,United States Of America,Central,Texas,Gainesville,76240,500000,60000
4,188,Mrs.,ALEX,NaN,harrell,Corporate,Single,Male,2/5/1997,12/31/2010,...,12/27/2036,3/31/2014,9/25/2014,United States Of America,Central,Texas,Gainesville,76240,500000,60000


In [ ]:
# List of columns to merge
cols_to_combine = ['Customer Title', 'Customer First Name', 'Customer Middle Name', 'Customer Last Name']

# 2. Get the index (position) of the first column to be replaced
# This tells us where to put the new column so it sits in the "vacated" space.
target_index = df0.columns.get_loc('Customer Title')

# 3. Create the combined values
# We generate this as a Series first, without attaching it to the df yet.
new_col_values = df0[cols_to_combine].apply(
    lambda x: ' '.join(x.dropna().astype(str)), 
    axis=1
)

# 4. Insert the new column at the target index
df0.insert(target_index, 'Customer Name', new_col_values)

# 5. Drop the old columns
df0 = df0.drop(columns=cols_to_combine)

print(df0)

In [ ]:
target_index = df1.columns.get_loc('Customer Title')

# 3. Create the combined values
# We generate this as a Series first, without attaching it to the df yet.
new_col_values = df1[cols_to_combine].apply(
    lambda x: ' '.join(x.dropna().astype(str)), 
    axis=1
)

# 4. Insert the new column at the target index
df1.insert(target_index, 'Customer Name', new_col_values)

# 5. Drop the old columns
df1 = df1.drop(columns=cols_to_combine)




In [22]:
cols_to_combine = ['Customer Title', 'Customer First Name', 'Customer Middle Name', 'Customer Last name']
target_index = df2.columns.get_loc('Customer Title')

# 3. Create the combined values
# We generate this as a Series first, without attaching it to the df yet.
new_col_values = df2[cols_to_combine].apply(
    lambda x: ' '.join(x.dropna().astype(str)), 
    axis=1
)

# 4. Insert the new column at the target index
df2.insert(target_index, 'Customer Name', new_col_values)

# 5. Drop the old columns
df2 = df2.drop(columns=cols_to_combine)

In [23]:
print(df2.columns.tolist())

['Customer ID', 'Customer Name', 'Customer_Segment', 'Maritial_Status', 'Gender', 'DOB', 'Effective_Start_Dt', 'Effective_End_Dt', 'Policy_Type_Id', 'Policy_Type', 'Policy_Type_Desc', 'Policy_Id', 'Policy_Name', 'Premium_Amt', 'Policy_Term', 'Policy_Start_Dt', 'Policy_End_Dt', 'Next_Premium_Dt', 'Actual_Premium_Paid_Dt', 'Country', 'Region', 'State or Province', 'City', 'Postal Code', 'Total_Policy_Amt', 'Premium_Amt_Paid_TillDate']


In [25]:
import pandas as pd

def clean_date_formats(df):
    """
    Identifies specific date columns and converts them to standard datetime objects.
    These are automatically compatible with SQL DATE/TIMESTAMP types.
    """
    # 1. List of date columns based on your schema
    date_cols = [
        'DOB', 
        'Effective_Start_Dt', 
        'Effective_End_Dt', 
        'Policy_Start_Dt', 
        'Policy_End_Dt', 
        'Next_Premium_Dt', 
        'Actual_Premium_Paid_Dt'
    ]
    
    # 2. Loop through and convert
    for col in date_cols:
        if col in df.columns:
            # Convert to datetime (handles most formats automatically)
            # errors='coerce' turns invalid text into NaT (SQL NULL)
            df[col] = pd.to_datetime(df[col], errors='coerce')

            # Optional: If you strictly need YYYY-MM-DD strings (for CSVs/Manual SQL writing)
            # uncomment the line below. Otherwise, keep as datetime objects (Recommended).
            # df[col] = df[col].dt.strftime('%Y-%m-%d')
            
    return df

In [26]:
# Usage
df2 = clean_date_formats(df2)
df1 = clean_date_formats(df1)
df0 = clean_date_formats(df0)

In [35]:
import pandas as pd

def enforce_strict_schema(df):
    """
    Replaces DataFrame column names with a strict target list based on position.
    Raises an error if the column counts do not match exactly.
    """
    
    # 1. Define the strict Target List
    target_columns = [
        'Customer_ID', 'Customer_Name', 'Customer_Segment', 'Maritial_Status',
        'Gender', 'DOB', 'Effective_Start_Dt', 'Effective_End_Dt',
        'Policy_Type_Id', 'Policy_Type', 'Policy_Type_Desc', 
        'Policy_Id', 'Policy_Name', 'Premium_Amt', 'Policy_Term', 
        'Policy_Start_Dt', 'Policy_End_Dt', 'Next_Premium_Dt', 'Actual_Premium_Paid_Dt', 
        'Country', 'Region', 'State_Province', 'City', 'Postal_Code', 
        'Total_Policy_Amt', 'Premium_Amt_Paid_TillDate'
    ]
    
    # 2. Strict Validation: Check size mismatch
    if len(df.columns) != len(target_columns):
        raise ValueError(
            f"❌ Column Count Mismatch!\n"
            f"Expected: {len(target_columns)} columns\n"
            f"Received: {len(df.columns)} columns\n"
            f"Please check if you have dropped/added extra columns in previous steps."
        )
    
    # 3. Replace columns (Blind positional assignment)
    df.columns = target_columns
    
    print("✅ Success: Columns renamed successfully.")
    return df

# --- Usage ---


In [36]:
try:
    # df2 is your input dataframe
    df2 = enforce_strict_schema(df2)
    print(df2.head())
except ValueError as e:
    print(e)

✅ Success: Columns renamed successfully.
   Customer_ID          Customer_Name Customer_Segment Maritial_Status Gender  \
0         1354       Mr. Aaron Dillon         Consumer        Divorced   Male   
1          697  Mister. Adam G Sawyer        Corporate       Remarried   Male   
2          674        Mr Albert Frost   Small Business         Married   Male   
3          188      Mrs. Alex Harrell        Corporate        Divorced   Male   
4         3106  Mr. Alexander O'Brien      Home Office       Remarried   Male   

         DOB Effective_Start_Dt Effective_End_Dt  Policy_Type_Id  Policy_Type  \
0 1997-02-05         2007-12-31              NaT           19183    Endowment   
1 1997-02-05         2011-06-30              NaT           19184    Moneyback   
2 1993-02-06         2009-12-31              NaT           19185    ChildPlan   
3 1997-02-05         2010-12-31              NaT           19186   UnitLinked   
4 1996-02-05         2007-12-31              NaT           19187  A

In [37]:
try:
    # df2 is your input dataframe
    df1 = enforce_strict_schema(df1)
    print(df1.head())
except ValueError as e:
    print(e)

✅ Success: Columns renamed successfully.
   Customer_ID                Customer_Name Customer_Segment Maritial_Status  \
0         1354             Mr. AARON DILLON         Consumer         Married   
1          697        Mister. aDAM G sawyer        Corporate        Divorced   
2          674              Mr aLBERT frosT   Small Business       Remarried   
3          188            Mrs. AlEx harrell        Corporate         Married   
4         3106  Mistress. AlexandeR O'Brien      Home Office        Divorced   

  Gender        DOB Effective_Start_Dt Effective_End_Dt  Policy_Type_Id  \
0   Male 1997-02-05         2007-12-31              NaT           19183   
1   Male 1997-02-05         2011-06-30              NaT           19184   
2   Male 1993-02-06         2009-12-31              NaT           19185   
3   Male 1997-02-05         2010-12-31              NaT           19186   
4   Male 1996-02-05         2007-12-31              NaT           19187   

   Policy_Type  ... Policy_

In [38]:
try:
    # df2 is your input dataframe
    df0 = enforce_strict_schema(df0)
    print(df0.head())
except ValueError as e:
    print(e)

✅ Success: Columns renamed successfully.
   Customer_ID          Customer_Name Customer_Segment Maritial_Status Gender  \
0         1354       Mr. AARON Dillon         Consumer          Single   Male   
1          697  Mister. Adam G Sawyer        Corporate         Married   Male   
2          674        Mr AlBERT FROST   Small Business        Divorced   Male   
3          188      Mrs. ALEX harrell        Corporate          Single   Male   
4          188      Mrs. ALEX harrell        Corporate          Single   Male   

         DOB Effective_Start_Dt Effective_End_Dt  Policy_Type_Id Policy_Type  \
0 1997-02-05         2007-12-31              NaT           19181    TermLife   
1 1997-02-05         2011-06-30              NaT           19182   WholeLife   
2 1993-02-06         2009-12-31              NaT           19183   Endowment   
3 1997-02-05         2010-12-31              NaT           19184   Moneyback   
4 1997-02-05         2010-12-31              NaT           19184   Money

In [39]:
# Standard export
df0.to_csv('Central_day0.csv', index=False)
df1.to_csv('Central_day1.csv', index=False)
df2.to_csv('Central_day2.csv', index=False)

# If you have special characters (accents, currency symbols) and use Excel:
# df2.to_csv('customer_data.csv', index=False, encoding='utf-8-sig')

In [45]:
def fill_missing_by_type(df, default_date='1900-01-01', default_text='Unknown', default_num=0):
    df = df.copy()

    for col in df.columns:
        # 1. Date Handling
        if is_datetime64_any_dtype(df[col]):
            df[col] = df[col].fillna(pd.Timestamp(default_date))
        
        # 2. Numeric Handling (The Critical Part)
        elif is_numeric_dtype(df[col]):
            # Fill the missing values first
            df[col] = df[col].fillna(default_num)
            
            # FORCE conversion to Integer if it's a whole number ID
            # This turns 1024.0 -> 1024
            try:
                df[col] = df[col].astype(int)
            except ValueError:
                # If conversion fails (e.g. column has real decimals like 10.50), keep as float
                pass
        
        # 3. String Handling
        elif is_string_dtype(df[col]) or is_object_dtype(df[col]):
            df[col] = df[col].fillna(default_text)
            
    return df

In [50]:
df0_cleaned = fill_missing_by_type(df0, default_num=0, default_text='N/A', default_date='1900-01-01')
df1_cleaned = fill_missing_by_type(df1, default_num=0, default_text='N/A', default_date='1900-01-01')
df2_cleaned = fill_missing_by_type(df2, default_num=0, default_text='N/A', default_date='1900-01-01')


In [51]:
df0_cleaned.to_csv('Central_day0.csv', index=False)
df1_cleaned.to_csv('Central_day1.csv', index=False)
df2_cleaned.to_csv('Central_day2.csv', index=False)

In [52]:
df0_cleaned['Policy_Id']

0      19181_1354
1       19182_697
2       19183_674
3       19184_188
4       19184_188
          ...    
319    98765_2954
320     98765_146
321     98765_146
322     13579_962
323    13579_2457
Name: Policy_Id, Length: 324, dtype: object